In [2]:
import matplotlib.pyplot as plt
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import io
import os
from glob import glob
from tqdm import tqdm



In [3]:
def parse_tensorboard_logs(logdir, img=False):
    event_acc = EventAccumulator(logdir)
    event_acc.Reload()

    # Get all tags (usually represent different summaries such as scalars, histograms, etc.)
    tags = event_acc.Tags()

    # Get scalar data
    scalar_data = {}
    for tag in tags['scalars']:
        scalar_data[tag] = [(event.step, event.value) for event in event_acc.Scalars(tag)]

    if img:
        # Get image data
        image_data = {}
        for tag in tags['images']:
            img_events = event_acc.Images(tag)
            image_data[tag] = []
            for img_event in img_events:
                step = img_event.step
                encoded_image = img_event.encoded_image_string
                # Decode the image string to PIL Image
                image = Image.open(io.BytesIO(encoded_image))
                image_data[tag].append((step, image))
    
        return scalar_data, image_data
    else:
        return scalar_data

In [4]:
def combine_plots(folder_list, k='loss/generator/train', skip=10**3):
    for folder in tqdm(folder_list):
        print(f'Parsing {folder}')
        scalar_data = parse_tensorboard_logs(folder)
        print(len(scalar_data[k]))
        plt.plot([x[1] for x in scalar_data[k][::skip]])

In [5]:

files = [] # Paths to logs

combine_plots(files)

0it [00:00, ?it/s]


# Draw samples

In [6]:
from labml_nn.gan.wasserstein.gradient_penalty.liperm.cifar10_experiment import Configs
from labml import experiment
from torchvision.utils import make_grid

/auto/home/tigrann/miniforge3/envs/liperm/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `InceptionScore` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)  # noqa: B028


In [ ]:
exp_name, run_uuid ='',  ''

experiment.create(name=exp_name)

# Create configs
conf = Configs()
# Load custom configuration of the training run
configs_dict = experiment.load_configs(run_uuid)

# Initialize
conf.init()

# Set PyTorch modules for saving and loading
experiment.add_pytorch_models({'generator': conf.generator,
                               'discriminator': conf.discriminator,
                               'inverse_generator': conf.inverse_generator})

# Load training experiment
experiment.load(run_uuid)

experiment.start()

In [ ]:
latent = conf.sample_z(25)
generated_images = (conf.generator(latent).detach().cpu() + 1 ) / 2
grid = make_grid(generated_images, nrow=5, pad_value=1.0).permute(1, 2, 0)

In [ ]:
plt.imshow(grid)
plt.axis('off');